In [1]:
import pandas as pd

# import docx
# import re
import nltk
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize

from dataset_prep import create_total_dataframe
from data_preprocessing import preprocess_text

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500000)

/Users/darinaponomarova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/darinaponomarova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/darinaponomarova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
all_docx = ["data/2022_satellite.docx", "data/2021_satellite.docx"]

# all_docx = ["data/2020_satellite.docx"]

total_df = create_total_dataframe(all_docx)


/Users/darinaponomarova/Study/DiplomaMasters/dataset_prep.py:54: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['Authors'], df['Article_Name'] = df['Article_Name'].str.rsplit('.', 1).str


In [3]:
total_df.head()

Article_Name  \
0                                                                Internet of Things Eco-System Concept for Healthy Eating   
1               Development of a Mathematical Model Determination of the Proximity of Vectors in the Project-Vector Space   
2  Investigation of the Algorithm of Near Search of the Compromise Median in the Problem of Collective Ranking of Objects   
3                                                                                      Analysis of Host Detection Methods   
4                                                             Symmetric Encryption of Messages Based on the Use of Images   

  Page  \
0    7   
1   11   
2   14   
3   18   
4   21   

                                                                          Authors  \
0                                                      Avramenko К., Latysheva Т.   
1  Biloshchytskyi A., Kuchansky A., Biloshchytska S., Andrashko Yu., Faizullin A.   
2                                                   Bovsunovska M., Hnatiienko H.   
3                                                            Buchyk S., Saroka S.   
4                                                            Buchyk S., Tsapro D.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [4]:
total_df['Processed_Text'] = total_df['Text'].apply(preprocess_text)

In [5]:
total_df.head()

Article_Name  \
0                                                                Internet of Things Eco-System Concept for Healthy Eating   
1               Development of a Mathematical Model Determination of the Proximity of Vectors in the Project-Vector Space   
2  Investigation of the Algorithm of Near Search of the Compromise Median in the Problem of Collective Ranking of Objects   
3                                                                                      Analysis of Host Detection Methods   
4                                                             Symmetric Encryption of Messages Based on the Use of Images   

  Page  \
0    7   
1   11   
2   14   
3   18   
4   21   

                                                                          Authors  \
0                                                      Avramenko К., Latysheva Т.   
1  Biloshchytskyi A., Kuchansky A., Biloshchytska S., Andrashko Yu., Faizullin A.   
2                                                   Bovsunovska M., Hnatiienko H.   
3                                                            Buchyk S., Saroka S.   
4                                                            Buchyk S., Tsapro D.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the processed text
tfidf_matrix = vectorizer.fit_transform(total_df['Processed_Text'])

# tfidf_matrix now contains the TF-IDF scores for your texts

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity
cosine_similarities = cosine_similarity(tfidf_matrix)

# cosine_similarities is a matrix where each cell [i,j] represents the cosine similarity between text i and text j

In [14]:
def report_plagiarism_results(similarity_matrix, articles, threshold=0.60):
    potential_plagiarism_cases = []
    for i in range(len(similarity_matrix)):
        for j in range(i+1, len(similarity_matrix[i])):  # Compare each pair only once
            if similarity_matrix[i][j] >= threshold:
                case_details = {
                    'Text 1': articles[i],
                    'Text 2': articles[j],
                    'Similarity Score': similarity_matrix[i][j]
                }
                potential_plagiarism_cases.append(case_details)
    return potential_plagiarism_cases

# Get potential plagiarism cases
plagiarism_cases = report_plagiarism_results(cosine_similarities, total_df['Article_Name'])

# Display the cases
for case in plagiarism_cases:
    print(f"Article '{case['Text 1']}' and Article '{case['Text 2']}' might be plagiarized with a similarity score of {case['Similarity Score']:.2f}")

Article 'Using Blockchain for Data Storage' and Article 'How Blockchain Technology Can Affect the Banking Industry' might be plagiarized with a similarity score of 0.65


In [11]:
total_df[total_df['Article_Name'] == 'Using Blockchain for Data Storage']

,Article_Name,Page,Authors,Text,Processed_Text
9,Using Blockchain for Data Storage,35,"Dakov S., Mohylevych V., Parkhomenko I.","Abstract. The paper considers the use of blockchain for data storage. The main problems in use and possible ways to solve them are considered. Introduction At the heart of the blockchain is a sequence of blocks, each of which carries a certain amount of information. This volume is limited by the technical implementation of the blockchain. The limit shows the maximum file size that can be uploaded to the blockchain. This is sufficient for storing transaction information, however, if you need to store a larger file, you need to look for another solution. Basic material The problem of limiting the maximum block size can be solved in several ways. The simplest of them implies: Splitting a file into segments smaller than the block size. Thus, even the largest file can be written to the blockchain with a small block size. Encryption of data in segments. This will allow you to store information in an open blockchain and be sure of its confidentiality. Distribution of segments on the blockchain network. Thanks to this, the file will be saved in an unchanged form as long as at least one user is synchronized with the blockchain. This approach is borrowed from torrent trackers, but it is not suitable for storing data using the blockchain, even if you remove the fees for creating transactions, for the following reasons: Recording of information in the blockchain is carried out using transactions, which, in turn, require confirmation. A large file can take several thousand transactions, that is, several hours or even days of processing. Information in the blockchain is immutable. Therefore, you cannot delete or modify unnecessary data. All the files that got into the network and their variations will forever remain in the blockchain and theoretically someone will be able to view them sooner or later. Immutability will lead to another problem – an avalanche of blockchain size. If information cannot be deleted, it will only accumulate, eventually making the size of the blockchain too large. Let's consider several examples of using blockchain to store information: BigChainDB cloud storage with huge volume and very fast transactions, it is built on the RethinkDB cluster and uses NoSQL mechanisms to store blocks, thanks to which it has high fault tolerance and bandwidth. BigchainDB is a scalable blockchain database. It is designed to combine the best properties of distributed databases and blockchain. Sia is a decentralized cloud storage platform secured by blockchain technology. Sia's storage network uses hard drive capacity around the world, which is used to create a more reliable and cheaper way to store data than traditional cloud storage providers. Sia has its own blockchain and utility token that ensures its operation - Siacoin. MaidSafe is a decentralized internet project. The concept of MaidSafe predates Bitcoin by several years, although it is currently still in beta. When the SAFE network is complete, it will work similarly to the TOR network, meaning that the distribution and access of Internet content will be done in a P2P core network. Conclusion So, there are various options for data storage, but when using the blockchain, it is important to remember that current technologies do not allow storing large amounts of information inside the block chain. Therefore, the blockchain in this industry is used as an intermediary and ledger that monitors compliance with the terms of the agreement on providing storage from one person to another.",paper consid use blockchain data storag main problem use possibl way solv consid introduct heart blockchain sequenc block carri certain amount inform volum limit technic implement blockchain limit show maximum file size upload blockchain suffici store transact inform howev need store larger file need look anoth solut basic materi problem limit maximum block size solv sever wa

In [12]:
total_df[total_df['Article_Name'] == 'How Blockchain Technology Can Affect the Banking Industry']

Article_Name Page  \
36  How Blockchain Technology Can Affect the Banking Industry  111   

                        Authors  \
36  Nakonechnyj V., Lutsenko V.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             